In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian

import var_optimise

# Lagrange by Rudy

See Rudy's note on optimizing the variance with Lagrange multiplier method

In [2]:
lih = HamiltonianSmall('LiH', 1.5)
beh2 = HamiltonianSmall('BeH2', 1.3)

h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

water_jw = Hamiltonian('H2O_STO3g_14qubits', 'jw')
water_parity = Hamiltonian('H2O_STO3g_14qubits', 'parity')
water_bk = Hamiltonian('H2O_STO3g_14qubits', 'bk')

ammonia_jw = Hamiltonian('NH3_STO3g_16qubits', 'jw')
ammonia_parity = Hamiltonian('NH3_STO3g_16qubits', 'parity')
ammonia_bk = Hamiltonian('NH3_STO3g_16qubits', 'bk')

In [3]:
#pr = lih.pauli_rep  #OK with iterative updates
#pr = beh2.pauli_rep #OK with iterative updates
pr = h2_jw.pauli_rep 
n = pr.num_qubits
energy, state = pr.ground()
β = var_optimise.optimal_beta(pr)
objective = var_optimise.objective(pr, β) # the cost function's value
print("beta:", β)
print("objective:", objective)

beta: {0: [0.32757412301293015, 0.3275741299636807, 0.3448517470233893], 1: [0.21055085428780332, 0.21055084776801927, 0.5788982979441775], 2: [0.18405433478621222, 0.18405433467209423, 0.6318913305416936], 3: [0.38574729976050853, 0.3857473022815767, 0.2285053979579148], 4: [0.3275741269135194, 0.3275741259190058, 0.34485174716747474], 5: [0.21055084685927808, 0.21055084939774857, 0.5788983037429734], 6: [0.18405433951461217, 0.18405433471804336, 0.6318913257673445], 7: [0.38574729986775314, 0.3857472944520303, 0.22850540568021646]}
objective: 27.966056715321546


In [4]:
def prod_inverse(Q, β):
    # prod_{j in supp(Q)} beta_{j, Q_j}^{-1}
    pauli_to_index = {'X': 0, 'Y': 1, 'Z': 2}
    prod = 1.0
    for i in range(len(Q)):
        if Q[i] != 'I':
            index = pauli_to_index[Q[i]]
            b = β[(len(Q)-1)-i][index]  # qiskit ordering: if i=0 then need to look up beta[n]
            prod *= b
    return prod**(-1)

def lagrange_restriction_numerator(i, p, dic, β):
    tally = 0.0
    for Q in dic:
        if Q[(len(Q)-1)-i] == p: # qiskit ordering
            tally += (dic[Q]**2) * prod_inverse(Q, β)
    return tally

def lagrange_restriction_denominator(i, dic, β):
    tally = 0.0
    for Q in dic:
        if Q[(len(Q)-1)-i] != "I": # qiskit ordering
            tally += (dic[Q]**2) * prod_inverse(Q, β)
    return tally

def lagrange_restriction(i, p, dic, β, objective):
    numerator = lagrange_restriction_numerator(i, p, dic, β)
    denominator = lagrange_restriction_denominator(i, dic, β)
    return numerator / denominator

In [5]:
#THIS IS  WHERE WE USE ITERATIVE UPDATES TO FIND THE BEST BETAS

import numpy as np

def distance(betas1, betas2):
    dist = 0.0
    for k in betas1.keys():
        dist += np.sqrt( np.sum( (np.array(betas1[k]) - np.array(betas2[k]))**2 ) )
    return dist

def update_betas(dic, nQbits, betas = None, pauli_to_index={"X":0, "Y":1, "Z":2}, weight=0.1):
    """
        iteratively update betas with new values by weight 
    """
    if betas is None: #initialize with random uniform
        betas = {}
        for i in range(nQbits):
            betas[i] = [ 1./3 for _ in range(3) ]
    newBetas = {}
    for i in range(nQbits):
        newBetas[i] = []
        for j, p in enumerate(("X", "Y", "Z")):
            newBetas[i].append( (1.- weight) * betas[i][j] + weight * lagrange_restriction(i, p, dic, betas, 1.0) )
    return newBetas, distance(newBetas, betas)

In [6]:
def test_iterative_updates(tol=1.0e-5):
    for name, molecules in  ( ("lih", lih) , ("beh2", beh2), ("h2_jw", h2_jw), ("h2_parity", h2_parity), \
                             ("h2_bk", h2_bk), ("water_jw", water_jw), ("water_parity", water_parity), \
                             ("water_bk", water_bk), ("ammonia_jw", ammonia_jw), ("ammonia_parity", ammonia_parity), \
                             ("ammonia_bk", ammonia_bk)):
        print("Molecule=", name, "------------------")
        pr = molecules.pauli_rep
        n = pr.num_qubits
        energy, state = pr.ground()
        β = var_optimise.optimal_beta(pr)
        objective = var_optimise.objective(pr, β) # the cost function's value
        # THIS IS THE EXPERIMENT WITH ITERATIVE UPDATES TO FIND BETAS 
        tol = 1.0e-5
        iter = 0
        oldBetas = None
        while True and iter < 1000:
            newBetas, error = update_betas(pr.dic, pr.num_qubits, oldBetas)
            oldBetas = newBetas
            print("\t", iter, error)
            #print(newBetas)
            iter += 1
            if error < tol:
                print("MOLECULE", name, "CONVERGE with error = ", error)
                break
        #break
        #print(newBetas)
        print("Distance from true values:", distance(newBetas, β)) 

In [7]:
test_iterative_updates()

Molecule= lih ------------------
	 0 0.22521544175668862
	 1 0.15642603210068023
	 2 0.10384632613187514
	 3 0.06743704224756017
	 4 0.0434029019342668
	 5 0.027845782586829227
	 6 0.017847076943401482
	 7 0.01143566515455247
	 8 0.00732879003701649
	 9 0.004707167156729455
	 10 0.003158354841244272
	 11 0.00263981203325602
	 12 0.0022883234017653624
	 13 0.001988390415836143
	 14 0.0017286525560218104
	 15 0.0015031605130427386
	 16 0.0013073000492835598
	 17 0.0011371663243359784
	 18 0.000989382960020178
	 19 0.0008610147402651559
	 20 0.0007495075318951059
	 21 0.0006526394740604469
	 22 0.0005684789204476818
	 23 0.0004953475656437186
	 24 0.0004317880039208444
	 25 0.0003765352119217345
	 26 0.0003284915317229276
	 27 0.00028670477291346146
	 28 0.00025034908376715676
	 29 0.00021870827084077658
	 30 0.0001911612752093888
	 31 0.00016716954193856927
	 32 0.00014626604683277506
	 33 0.0001280457704713488
	 34 0.00011215743371917649
	 35 9.829633104673758e-05
	 36 8.619811806004216

	 26 0.015530435647641809
	 27 0.014013023520017922
	 28 0.012641406266815024
	 29 0.011401950837363525
	 30 0.010282251860617127
	 31 0.009271023538501831
	 32 0.008358002427317473
	 33 0.007533859241956579
	 34 0.006790118369086496
	 35 0.006119084130807645
	 36 0.005513773075005323
	 37 0.004967851725670055
	 38 0.004475579332302944
	 39 0.00403175522926104
	 40 0.003631670465042838
	 41 0.0032710633958752252
	 42 0.0029460789628607594
	 43 0.0026532313909658524
	 44 0.002389370063619054
	 45 0.002151648340185066
	 46 0.0019374950959781115
	 47 0.0017445887763519092
	 48 0.0015708337679898546
	 49 0.001414338901990078
	 50 0.0012733979146400897
	 51 0.0011464717029500333
	 52 0.0010321722229334526
	 53 0.0009292478892632464
	 54 0.000836570345209933
	 55 0.0007531224816239256
	 56 0.0006779875931307817
	 57 0.0006103395686244301
	 58 0.0005494340215314712
	 59 0.0004946002732249246
	 60 0.00044523411030630166
	 61 0.0004007912433504279
	 62 0.0003607814010471072
	 63 0.0003247629995

/Users/rraymondhp/miniconda3/envs/qiskit_keio/lib/python3.6/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


	 0 0.648627862193856
	 1 0.5963681614691156
	 2 0.5280087807008399
	 3 0.4595688784329707
	 4 0.3968834547982579
	 5 0.3411703360993533
	 6 0.2924191940081949
	 7 0.2502292868270797
	 8 0.21402070706639434
	 9 0.18312234113173056
	 10 0.15683753943716078
	 11 0.13449638827675844
	 12 0.1154903664992563
	 13 0.09928914191602532
	 14 0.08544347255016269
	 15 0.07357947522023017
	 16 0.0633887762282915
	 17 0.054617609797722295
	 18 0.0470565867701627
	 19 0.040531898117164906
	 20 0.03489814012462611
	 21 0.030032650061185794
	 22 0.025831118570968602
	 23 0.022204219886721197
	 24 0.01907502304878393
	 25 0.016376987503828416
	 26 0.014052389361871125
	 27 0.012051063055725557
	 28 0.010329374604443381
	 29 0.008849366922753458
	 30 0.007578035500791008
	 31 0.00648670552185087
	 32 0.005550490308517233
	 33 0.004747816947480711
	 34 0.004060008887862479
	 35 0.0034709178789461064
	 36 0.0029665992768690835
	 37 0.002535025831697986
	 38 0.002165835786772473
	 39 0.001850111625302981
	

KeyboardInterrupt: 

## Tester codes (OLD)

In [ ]:
# THIS IS THE EXPERIMENT WITH ITERATIVE UPDATES TO FIND BETAS 
tol = 1.0e-5
iter = 0
oldBetas = None
while True and iter < 1000:
    newBetas, error = update_betas(pr.dic, pr.num_qubits, oldBetas)
    oldBetas = newBetas
    print(iter, error)
    #print(newBetas)
    iter += 1
    if error < tol:
        print("\nCONVERGE with error = ", error)
        print("\n")
        break
print(newBetas)
print("Distance from true values:", distance(newBetas, β))

In [ ]:
lagrange_restriction(0, 'X', pr.dic, β, objective)

In [ ]:
lagrange_restriction(0, 'Y', pr.dic, β, objective)

In [ ]:
lagrange_restriction(0, 'Z', pr.dic, β, objective)